In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
import os
import tensorflow as tf
import numpy as np
import urllib.request as req
import shutil
from tqdm import tqdm
from PIL import Image

In [3]:
SOURCE_URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

In [4]:
DESTINATION = "data"
os.makedirs(DESTINATION, exist_ok=True)
data_file="data.zip"
DESTINATION_ZIP_PATH = os.path.join(DESTINATION,data_file)

In [5]:
DESTINATION_ZIP_PATH

In [6]:
pwd

In [7]:
filename,headers = tqdm(req.urlretrieve(SOURCE_URL,DESTINATION_ZIP_PATH))

In [8]:
filename

In [9]:
print(headers)

In [10]:
from zipfile import ZipFile
with ZipFile(filename,'r') as zip_file:
    zip_file.extractall("data/")

In [11]:
TARGET_DATA_DIR = ["Cat","Dog"]
#PARENT_DIR ="data/PetImages" for linux
PARENT_DIR =os.path.join("data","PetImages") # for all os

In [12]:
for dirs in os.listdir(PARENT_DIR):
    print(dirs)

In [13]:
for dirs in os.listdir(PARENT_DIR):
    print(os.path.join(PARENT_DIR,dirs))

In [14]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR,dirs)
    for imgs in os.listdir(full_path_data_dir):
        print(dirs,imgs)
        

In [15]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR,dirs)
    for imgs in os.listdir(full_path_data_dir):
        print(os.path.join(full_path_data_dir,imgs))

In [16]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR,dirs)
    for imgs in os.listdir(full_path_data_dir):
        path_to_img = os.path.join(full_path_data_dir,imgs)
        try:
            img=Image.open(path_to_img)
            img.verify()
            #print(f"{path_to_img } is verified")
        
        except Exception as e:
            print(f"{path_to_img } is bad")
            
            

In [17]:
BAD_DATA_DIR = "bad_data"
os.makedirs(BAD_DATA_DIR,exist_ok=True)

In [18]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR,dirs)
    for imgs in os.listdir(full_path_data_dir):
        path_to_img = os.path.join(full_path_data_dir,imgs)
        try:
            img=Image.open(path_to_img)
            img.verify()
            #print(f"{path_to_img } is verified")
        
        except Exception as e:
            print(f"{path_to_img } is bad")
            bad_data_path=os.path.join(BAD_DATA_DIR,imgs)
            shutil.move(path_to_img,bad_data_path)
            
            

In [19]:
IMG_SIZE = (180, 180)
BATCH_SIZE = 32

In [20]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

In [21]:
### Create a log dir for tensorboard logs

import time

def get_log_path(base_log_dir=os.path.join("logs","fit")):
    uniqueName = time.asctime().replace(" ", "_").replace(":", "")
    log_path = os.path.join(base_log_dir, uniqueName)
    print(f"saving logs at: {log_path}")
    return log_path

log_dir = get_log_path()

In [22]:
file_writer = tf.summary.create_file_writer(logdir=log_dir)

In [23]:
for images, labels in train_ds.take(1):
    print(images.shape, labels)

In [24]:
images.shape

In [25]:
with file_writer.as_default():
    images = np.reshape(images[10:30],(-1, 180, 180, 3)) ### (20, 28, 28, 1)

    tf.summary.image("samples",images, max_outputs=25, step=0)

In [26]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir research/logs/fit